# This is the 4th notebook

The goal right now is to do several things
1. Remove images from our data set since there is no real reason to hold them there. Just slows down
2. Look over over labelling function
3. Don't forget to pickle
## 4. Think

Actually, let's also not simply copy and paste everything everywhere but instead think

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display, Image
from pigeon import annotate
import random

random.seed(82)
%matplotlib inline

In [2]:
df = pd.read_pickle("./df_cleaned.pkl")

In [3]:
df.head()

,video_id,img
0,J_CFBjAyPWE,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
1,5qxqo2CQakA,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
2,4Syrtc5aRBY,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
3,1mpdJcGj8Qg,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
4,coOKvrsmQiI,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."


In case you were following notebook by notebook, in the last notebook we did not commit any changes which is why we dont yet have labeling column

We also have the img column, but we don't really need to have all of them loaded and displayed, instead we can just load one by one

In [10]:
df.drop("img", axis=1, inplace=True)
df['label'] = None

In [11]:
df.head()

,video_id,label
0,J_CFBjAyPWE,None
1,5qxqo2CQakA,None
2,4Syrtc5aRBY,None
3,1mpdJcGj8Qg,None
4,coOKvrsmQiI,None


In [12]:
df.columns

Index(['video_id', 'label'], dtype='object')

In [16]:
#from pigeon import annotate
#from IPython.display import display, Image

annotations = annotate(
  ['data/imgs/J_CFBjAyPWE.jpg', 'data/imgs/J_CFBjAyPWE.jpg', 'data/imgs/5qxqo2CQakA.jpg'],
  options=['Clickbait', 'Not_Clickbait', 'Maybe'],
  display_fn=lambda filename: display(Image(filename))
)

HTML(value='0 examples annotated, 4 examples left')

Output()

In [15]:
help(annotate)

Help on function annotate in module pigeon.annotate:

annotate(examples, options=None, shuffle=False, include_skip=True, display_fn=<function display at 0x104dea170>)
    Build an interactive widget for annotating a list of input examples.
    
    Parameters
    ----------
    examples: list(any), list of items to annotate
    options: list(any) or tuple(start, end, [step]) or None
             if list: list of labels for binary classification task (Dropdown or Buttons)
             if tuple: range for regression task (IntSlider or FloatSlider)
             if None: arbitrary text input (TextArea)
    shuffle: bool, shuffle the examples before annotating
    include_skip: bool, include option to skip example while annotating
    display_fn: func, function for displaying an example to the user
    
    Returns
    -------
    annotations : list of tuples, list of annotated examples (example, label)



In [80]:
df_test = df[:10] # Always test first

def label(df, n):
    # n is the number of how many you want to label
    temp = df[df['label'].isna()][:n]
    ids = temp['video_id']
    
    annotations = annotate(
      ids,
      options=['yes', 'no', 'Maybe'],
      display_fn=lambda filename: display(Image(f'data/imgs/{filename}.jpg'))
    )
    
    return annotations
    #df[df['video_id'] == ids] = annotations
    

ans = label(df_test, 5)

HTML(value='0 examples annotated, 6 examples left')

Output()

Annotation done.


In [82]:
dict(ans)

{'J_CFBjAyPWE': 'no',
 '5qxqo2CQakA': 'yes',
 '4Syrtc5aRBY': 'no',
 '1mpdJcGj8Qg': 'no',
 'coOKvrsmQiI': 'yes'}

Now the finding a way of actually assigning these values back into our original df turned out to be a torture, mainly because I couldn't not formulate a good question to ask (google). 

'how to fill in values in a column'

'pandas fill in custom values based on dictionary'

etc. 

When you get stuck like this the only last hope (before writing the ugliest code of your life) is not simply looking through the StackOverflow but actually writing a quesiton, which turned out to be harder but still doable. Here is the link for curious: 
https://stackoverflow.com/questions/60240032/how-to-assign-custom-values-to-specific-columns-using-pandas?noredirect=1#comment106554262_60240032
If you think it looks fairly simple, well, that's because I had to edit it several times (or maybe 2-digit iq, who knows)

In [83]:
# STACK OVERFLOW QUESITON GENERATOR
import pandas as pd

so = pd.DataFrame({'id': ['A', 'B', 'C', 'D', 'E', 'F'], 
                   'label': ['yes', 'no', None, None, None, 'yes']})
new_labels = {'D':'yes', 'E':'no'}


# part where I insert new_labels into so

In [84]:
so

,id,label
0,A,yes
1,B,no
2,C,None
3,D,None
4,E,None
5,F,yes


In the summary, the answer can be found here: https://stackoverflow.com/questions/20250771/remap-values-in-pandas-column-with-a-dict

```
df['label'] = df['id'].map(new_labels).fillna(df['label'])
```

In [85]:
df_test

,video_id,label
0,J_CFBjAyPWE,None
1,5qxqo2CQakA,None
2,4Syrtc5aRBY,None
3,1mpdJcGj8Qg,None
4,coOKvrsmQiI,None
5,_Ch5ClOB9AE,None
6,x-uBFJLY8Ow,None
7,QDSEpjjavhY,None
8,AHIrklVaooQ,None
9,0q9Cm11d-fE,None


In [87]:
df_test['label'] = df_test['video_id'].map(dict(ans)).fillna(df_test['label'])

/Users/ramil/Development/youtube_trending_2020/project_youtube/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [88]:
df_test

,video_id,label
0,J_CFBjAyPWE,no
1,5qxqo2CQakA,yes
2,4Syrtc5aRBY,no
3,1mpdJcGj8Qg,no
4,coOKvrsmQiI,yes
5,_Ch5ClOB9AE,None
6,x-uBFJLY8Ow,None
7,QDSEpjjavhY,None
8,AHIrklVaooQ,None
9,0q9Cm11d-fE,None


In [89]:
df

,video_id,label
0,J_CFBjAyPWE,no
1,5qxqo2CQakA,yes
2,4Syrtc5aRBY,no
3,1mpdJcGj8Qg,no
4,coOKvrsmQiI,yes
...,...,...
5957,b191evqWAwE,None
5958,Nk5yOFCsNeE,None
5959,iN8zmmCRqUA,None
5960,ahAnbeyxlic,None


Ouch, for some reason .copy() doesn't copy as I wanted, so out original df changed. Let's clean it

In [90]:
df['label'] = None

Okay so now we have a way to fill in our data, and we can finally also store it at the very end

Since we now have found all of the components that we need, let's pickle what we have and start a new notebook

In [94]:
df.to_pickle("./df_no_img_no_labels.pkl")